<a href="https://colab.research.google.com/github/Sekoudioum/Dioumessy_ML/blob/main/Projet_6_el_karaoui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

In [39]:
df=pd.read_csv('/content/drive/MyDrive/thyroidDF.csv')

In [69]:
! git clone https://github.com/Sekoudioum/Dioumessy_ML.git

Cloning into 'Dioumessy_ML'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [40]:
df.head()

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,NaN,f,NaN,f,NaN,f,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,128.0,f,NaN,f,NaN,f,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047


# Nouvelle section

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9172 entries, 0 to 9171
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  9172 non-null   int64  
 1   sex                  8865 non-null   object 
 2   on_thyroxine         9172 non-null   object 
 3   query_on_thyroxine   9172 non-null   object 
 4   on_antithyroid_meds  9172 non-null   object 
 5   sick                 9172 non-null   object 
 6   pregnant             9172 non-null   object 
 7   thyroid_surgery      9172 non-null   object 
 8   I131_treatment       9172 non-null   object 
 9   query_hypothyroid    9172 non-null   object 
 10  query_hyperthyroid   9172 non-null   object 
 11  lithium              9172 non-null   object 
 12  goitre               9172 non-null   object 
 13  tumor                9172 non-null   object 
 14  hypopituitary        9172 non-null   object 
 15  psych                9172 non-null   o

In [42]:
df.isna().sum().sort_values()

age                       0
referral_source           0
TBG_measured              0
FTI_measured              0
T4U_measured              0
TT4_measured              0
T3_measured               0
TSH_measured              0
target                    0
hypopituitary             0
tumor                     0
goitre                    0
psych                     0
query_hyperthyroid        0
on_thyroxine              0
query_on_thyroxine        0
on_antithyroid_meds       0
lithium                   0
sick                      0
patient_id                0
pregnant                  0
thyroid_surgery           0
I131_treatment            0
query_hypothyroid         0
sex                     307
TT4                     442
FTI                     802
T4U                     809
TSH                     842
T3                     2604
TBG                    8823
dtype: int64

In [43]:
cat_var= df.select_dtypes(include='object').columns

In [44]:
cat_var

Index(['sex', 'on_thyroxine', 'query_on_thyroxine', 'on_antithyroid_meds',
       'sick', 'pregnant', 'thyroid_surgery', 'I131_treatment',
       'query_hypothyroid', 'query_hyperthyroid', 'lithium', 'goitre', 'tumor',
       'hypopituitary', 'psych', 'TSH_measured', 'T3_measured', 'TT4_measured',
       'T4U_measured', 'FTI_measured', 'TBG_measured', 'referral_source',
       'target'],
      dtype='object')

In [45]:
num_var=df.select_dtypes(exclude= 'object').columns

In [46]:
num_var

Index(['age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI', 'TBG', 'patient_id'], dtype='object')

In [47]:
#handling missing values in  both numerical and categorical columns
#1ère methode

df[num_var].isnull().sum()

age              0
TSH            842
T3            2604
TT4            442
T4U            809
FTI            802
TBG           8823
patient_id       0
dtype: int64

In [48]:
df[num_var]= df[num_var].fillna(value=0)

In [49]:
df[num_var].isnull().sum()

age           0
TSH           0
T3            0
TT4           0
T4U           0
FTI           0
TBG           0
patient_id    0
dtype: int64

In [50]:
df.target.value_counts()

-      6771
K       436
G       359
I       346
F       233
R       196
A       147
L       115
M       111
N       110
S        85
GK       49
AK       46
J        30
B        21
MK       16
Q        14
O        14
C|I      12
KJ       11
GI       10
H|K       8
D         8
FK        6
C         6
P         5
MI        2
LJ        1
GKJ       1
OI        1
D|R       1
E         1
Name: target, dtype: int64

In [51]:
df[cat_var].isnull().sum()

sex                    307
on_thyroxine             0
query_on_thyroxine       0
on_antithyroid_meds      0
sick                     0
pregnant                 0
thyroid_surgery          0
I131_treatment           0
query_hypothyroid        0
query_hyperthyroid       0
lithium                  0
goitre                   0
tumor                    0
hypopituitary            0
psych                    0
TSH_measured             0
T3_measured              0
TT4_measured             0
T4U_measured             0
FTI_measured             0
TBG_measured             0
referral_source          0
target                   0
dtype: int64

In [52]:
df['sex']=df['sex'].fillna(df['sex'].value_counts().index[0])

In [53]:
df[cat_var].isnull().sum()

sex                    0
on_thyroxine           0
query_on_thyroxine     0
on_antithyroid_meds    0
sick                   0
pregnant               0
thyroid_surgery        0
I131_treatment         0
query_hypothyroid      0
query_hyperthyroid     0
lithium                0
goitre                 0
tumor                  0
hypopituitary          0
psych                  0
TSH_measured           0
T3_measured            0
TT4_measured           0
T4U_measured           0
FTI_measured           0
TBG_measured           0
referral_source        0
target                 0
dtype: int64

In [54]:
df.isnull().sum()

age                    0
sex                    0
on_thyroxine           0
query_on_thyroxine     0
on_antithyroid_meds    0
sick                   0
pregnant               0
thyroid_surgery        0
I131_treatment         0
query_hypothyroid      0
query_hyperthyroid     0
lithium                0
goitre                 0
tumor                  0
hypopituitary          0
psych                  0
TSH_measured           0
TSH                    0
T3_measured            0
T3                     0
TT4_measured           0
TT4                    0
T4U_measured           0
T4U                    0
FTI_measured           0
FTI                    0
TBG_measured           0
TBG                    0
referral_source        0
target                 0
patient_id             0
dtype: int64

In [55]:
#Methode2 
# features_cat=df [cat_var].values
# features_num=df[num_var].values
# labels=df['target'].values
# X_train_cat, X_test_cat, y_train , y_test= train_test_split(features_cat, labels, test_size=0.2) 
# X_train_num , X_test_num , y_train , y_test= train_test_split(features_num, labels, test_size=0.2)

# X_num_imputer= SimpleImputer()
# X_cat_imputer= SimpleImputer(strategy='most_frequent')

# X_train_num = X_num_imputer.fit_transform(X_train_num)
# X_test_num =X_num_imputer.transform(X_test_num)

# X_train_cat= X_cat_imputer.fit_transform(X_train_cat)
# X_test_cat= X_cat_imputer.transform(X_test_cat)


# X_train= np.append(X_train_num, X_train_cat, axis=1)
# X_test=np.append(X_test_num, X_test_cat, axis=1)

In [56]:
# X_train= pd.DataFrame(X_train)
# X_test=pd.DataFrame(X_test)
# print(X_train.isna().sum().sort_values())
# print(X_test.isna().sum().sort_values())

In [57]:
#one hot encoding

col_to =['sex', 'on_thyroxine', 'query_on_thyroxine', 'on_antithyroid_meds',
       'sick', 'pregnant', 'thyroid_surgery', 'I131_treatment',
       'query_hypothyroid', 'query_hyperthyroid', 'lithium', 'goitre', 'tumor',
       'hypopituitary', 'psych', 'TSH_measured', 'T3_measured', 'TT4_measured',
       'T4U_measured', 'FTI_measured', 'TBG_measured', 'referral_source'
    ]
  
df_dummies= pd.get_dummies(df, columns=col_to, drop_first= True)
#df_dummies= pd.concat([df, df_dummies], axis=1)
#df_dummies.drop(labels= col_to, axis=1, inplace=True)


In [58]:
df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9172 entries, 0 to 9171
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   age                    9172 non-null   int64  
 1   TSH                    9172 non-null   float64
 2   T3                     9172 non-null   float64
 3   TT4                    9172 non-null   float64
 4   T4U                    9172 non-null   float64
 5   FTI                    9172 non-null   float64
 6   TBG                    9172 non-null   float64
 7   target                 9172 non-null   object 
 8   patient_id             9172 non-null   int64  
 9   sex_M                  9172 non-null   uint8  
 10  on_thyroxine_t         9172 non-null   uint8  
 11  query_on_thyroxine_t   9172 non-null   uint8  
 12  on_antithyroid_meds_t  9172 non-null   uint8  
 13  sick_t                 9172 non-null   uint8  
 14  pregnant_t             9172 non-null   uint8  
 15  thyr

In [59]:
features= df_dummies.drop('target', axis=1).values
labels=df_dummies['target'].values
X_train, X_test, y_train, y_test= train_test_split(features, labels, test_size=0.2)

In [60]:
#Scaling
scaler=StandardScaler()
X_train_scaled= scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [61]:
X_train_scaled.shape[0], y_train.shape[0]

(7337, 7337)

# Models **Evaluation**

In [62]:
models=[('logistic regression', LogisticRegression()), 
        ('Decision tree', DecisionTreeClassifier()),
        ('Random Forest', RandomForestClassifier()),
        ('Linear SVM', LinearSVC()),
        ('SGD classifier', SGDClassifier(loss='hinge'))]
for name, model in models:
  model.fit(X_train_scaled, y_train)
  preds=model.predict(X_test_scaled)
  accuracy= float(np.sum(preds==y_test)/y_test.shape[0])

  print('{} : {:.2f}'.format(name, accuracy))


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


logistic regression : 0.85
Decision tree : 0.94
Random Forest : 0.94


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Linear SVM : 0.80
SGD classifier : 0.82


# **KNN**

In [63]:
#autres manières de faire
knn= KNeighborsClassifier()
knn.fit(X_train_scaled, y_train)
knn_predict= knn.predict(X_test_scaled)

knn_train_accuracy= knn.score(X_train_scaled, y_train)
knn_test_accuracy= accuracy_score(y_test, knn_predict)
print('Training Accuracy :', knn_train_accuracy)
print('Model Accuracy Score {:2f}', knn_test_accuracy)
#print('confusion matrix \n', confusion_matrix(y_test, knn_predict))
print('classification report \n', classification_report(y_test, knn_predict))


Training Accuracy : 0.8611148970969061
Model Accuracy Score {:2f} 0.8119891008174387
classification report 
               precision    recall  f1-score   support

           -       0.83      0.98      0.90      1333
           A       0.71      0.50      0.59        30
          AK       0.62      0.50      0.56        10
           B       0.00      0.00      0.00         5
           C       0.00      0.00      0.00         1
         C|I       0.00      0.00      0.00         1
           D       0.50      0.50      0.50         2
           F       0.89      0.69      0.78        48
          FK       0.00      0.00      0.00         1
           G       0.73      0.15      0.26        71
          GI       0.00      0.00      0.00         1
          GK       1.00      0.14      0.25         7
         H|K       0.00      0.00      0.00         1
           I       0.75      0.46      0.57        78
           J       1.00      0.12      0.22         8
           K       0.74   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# `Logistic Regression`

In [64]:
logreg= LogisticRegression()
logreg.fit(X_train_scaled, y_train)
logreg_predict= logreg.predict(X_test_scaled)
 
logreg_training_accuracy= logreg.score(X_train_scaled, y_train)
logreg_testing_accuracy= accuracy_score(y_test, logreg_predict)

print('Training Accuracy{:2f}'.format(logreg_training_accuracy))
print('Model Accuracy Score {:2f}'.format(logreg_testing_accuracy))

print('classification report \n', classification_report(y_test, logreg_predict))


Training Accuracy0.873518
Model Accuracy Score 0.849591
classification report 
               precision    recall  f1-score   support

           -       0.87      0.98      0.92      1333
           A       0.78      0.60      0.68        30
          AK       0.57      0.40      0.47        10
           B       0.00      0.00      0.00         5
           C       0.00      0.00      0.00         1
         C|I       0.00      0.00      0.00         1
           D       1.00      0.50      0.67         2
           F       0.85      0.85      0.85        48
          FK       0.00      0.00      0.00         1
           G       0.89      0.45      0.60        71
          GI       0.00      0.00      0.00         1
          GK       1.00      0.29      0.44         7
         H|K       0.00      0.00      0.00         1
           I       0.62      0.51      0.56        78
           J       0.00      0.00      0.00         8
           K       0.83      0.66      0.73        96
 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0

# Random Forest


In [65]:
rf= RandomForestClassifier()
rf.fit(X_train_scaled, y_train)
rf_predict= rf.predict(X_test_scaled)

rf_training_accuracy= rf.score(X_train_scaled, y_train)
rf_testing_accuracy= accuracy_score(y_test, rf_predict)

print('Training Accuracy{:2f}'.format(rf_training_accuracy))
print('Model Accuracy Score {:2f}'.format(rf_testing_accuracy))

print('classification report \n', classification_report(y_test, rf_predict))


Training Accuracy1.000000
Model Accuracy Score 0.935150
classification report 
               precision    recall  f1-score   support

           -       0.95      0.99      0.97      1333
           A       0.88      0.73      0.80        30
          AK       0.91      1.00      0.95        10
           B       0.00      0.00      0.00         5
           C       0.00      0.00      0.00         1
         C|I       0.00      0.00      0.00         1
           D       1.00      1.00      1.00         2
           F       0.89      0.98      0.93        48
          FK       0.00      0.00      0.00         1
           G       0.95      1.00      0.97        71
          GI       1.00      1.00      1.00         1
          GK       1.00      0.86      0.92         7
         H|K       1.00      1.00      1.00         1
           I       0.81      0.67      0.73        78
           J       0.00      0.00      0.00         8
           K       0.96      0.85      0.91        96
 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**XGBClassifier**

In [66]:
xgb_cl= xgb.XGBClassifier(objective='reg: logistic', n_estimators=100, seed=123)
xgb_cl.fit(X_train_scaled, y_train)
xgb_predict= xgb_cl.predict(X_test_scaled)

xgb_training_accuracy = xgb_cl.score(X_train_scaled, y_train)
xgb_testing_accuracy = accuracy_score(y_test, xgb_predict)

print('Training Accuracy {:2f}'.format(xgb_training_accuracy))
print('Model Accuracy Score {:2f}'.format(xgb_testing_accuracy))

print('classification report \n ', classification_report(y_test, xgb_predict))



Training Accuracy 0.979010
Model Accuracy Score 0.942779
classification report 
                precision    recall  f1-score   support

           -       0.96      0.98      0.97      1333
           A       0.88      0.73      0.80        30
          AK       0.75      0.90      0.82        10
           B       0.00      0.00      0.00         5
           C       1.00      1.00      1.00         1
         C|I       0.00      0.00      0.00         1
           D       1.00      1.00      1.00         2
           F       0.94      0.94      0.94        48
          FK       0.50      1.00      0.67         1
           G       0.97      1.00      0.99        71
          GI       0.50      1.00      0.67         1
          GK       1.00      0.86      0.92         7
         H|K       1.00      1.00      1.00         1
           I       0.76      0.65      0.70        78
           J       0.88      0.88      0.88         8
           K       0.99      0.94      0.96        96

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [67]:
models= pd.DataFrame({
    'Models': ['KNeighbors Classifier', 'Logistic Regression', 'XGBoost Classifier',
                'Random Forest'],
    'Training_Accuracy': [knn_train_accuracy, logreg_training_accuracy, xgb_training_accuracy, rf_training_accuracy],
    'Model_Accuracy_Score': [ knn_test_accuracy, logreg_testing_accuracy, xgb_testing_accuracy, rf_testing_accuracy]

})
                   

In [68]:
models.sort_values(by='Model_Accuracy_Score', ascending=False)

,Models,Training_Accuracy,Model_Accuracy_Score
2,XGBoost Classifier,0.979010,0.942779
3,Random Forest,1.000000,0.935150
1,Logistic Regression,0.873518,0.849591
0,KNeighbors Classifier,0.861115,0.811989
